<a href="https://colab.research.google.com/github/TurkuNLP/Deep_Learning_in_LangTech_course/blob/master/bert_model_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic BERT operations


In [ ]:
!pip3 -q install datasets transformers

In [ ]:
import transformers
import datasets
import torch

In [ ]:
tokenizer=transformers.AutoTokenizer.from_pretrained("bert-base-cased") #you can also use the trusty "TurkuNLP/bert-base-finnish-cased-v1"

In [ ]:
# We will be running the model directly, so let's use return_tensors="pt" to get torch tensors rather than Python lists
texts=["Dogs like to [MASK] cats. They taste good.","Bad joke!"]
t=tokenizer(texts,padding=True, truncation=True, return_tensors="pt")
print("Input ids",t["input_ids"])
print("Token type ids",t["token_type_ids"])
print("Attention mask",t["attention_mask"])

In [ ]:
# This is what the first sequence looks like
tokenizer.decode(t["input_ids"][0])

# BERT: bare model
* How to use the bare model
* What does it give us?

In [ ]:
bert=transformers.AutoModel.from_pretrained("bert-base-cased") #"TurkuNLP/bert-base-finnish-cased-v1" if you run this in Finnish


* in torch the model's forward() function tends to be mapped to `__call__()` i.e. it is used when you call the model as if it were a function


In [ ]:
bert_out=bert(
    input_ids=t["input_ids"],
    attention_mask=t["attention_mask"],
    token_type_ids=t["token_type_ids"])
#an easy way to say the above would be bert(**t)


that's it, this is how you call BERT, now let's see what it gave us (not hard to figure out it is really a dictionary)

In [ ]:
bert_out.keys()

* last_hidden_state: the last layer of the encoder
* pooler_output: the `tanh` layer on top of `[CLS]`

In [ ]:
# Before you run this, stop to think:
# What will the shape be? How many dimensions? 1? 2? 3? more? And their approximate sizes?
# make a guess, see if it matches
bert_out.last_hidden_state.shape

In [ ]:
# And here? What will the shape be?
bert_out.pooler_output.shape

# BERT: masked language modelling output

* Not much we can do with the above
* But BERT is trained to predict masked words, let's try!

In [ ]:
# Have a look at HuggingFace automodels documentation to see what types of automodels there are
# (Do you notice how the complaint of unused weights got a lot shorter? Why do you think that is?)
bert=transformers.AutoModelForPreTraining.from_pretrained("bert-base-cased")

In [ ]:
# Tell the model it is not really being trained (disables dropout for example)
# I do not think this is needed but am playing it safe, the docs say it is put to eval mode upon load: https://huggingface.co/docs/transformers/main_classes/model#transformers.PreTrainedModel.from_pretrained.config
bert=bert.eval()

Now we can again run the model, and we will see the output is quite different!

In [ ]:
bert_out=bert(**t)
bert_out.keys()

In [ ]:
#What do you think these shapes will be?
print("Logits",bert_out["prediction_logits"].shape)
print("Seq relationship logits",bert_out["seq_relationship_logits"].shape)

In [ ]:
#cross-check
tokenizer.vocab_size

...now let's see how well this works for the masked word prediction...
* we need to find the most likely predicted words
* which can be achieved by arg-sorting the predictions and picking top N words
* this is easy and we have done this kind of stuff before
* now let's try straight in torch without a roundtrip to numpy

In [ ]:
top20=torch.argsort(predictions,dim=2,descending=True)[:,:,:20] #why dim=2? what does [:,:,:20] do?
print(top20)

In [ ]:
print(texts[0])
print("Guesses:",tokenizer.decode(top20[0,4]))

# ...in one block...

In [ ]:
texts=["Dogs like to [MASK] cats. They are cute."]
t=tokenizer(texts,padding=True, truncation=True, return_tensors="pt")
bert_out=bert(**t)
top20=torch.argsort(bert_out["prediction_logits"],dim=2,descending=True)[:,:,:20]
print("Guesses:",tokenizer.decode(top20[0,4]))

In [ ]:
print(t)
print(tokenizer.mask_token_id)

# TASKS

As an exercise, you can try to solve any subset of the following:

1. How good is BERT at the masked language modelling (MLM) task? Feed random texts e.g. from the IMDB dataset, mask a random token at a time, and check: did BERT predict it correctly?
2. If you did (1), can you answer did BERT predict it correctly in top-5?
3. Try can you do better. Make yourself a program which picks random texts from one of the datasets we used in this course and produces two files: one with segments of texts with one [MASK] and one with the correct answers. Then try to guess the words without looking at the latter file and then compare your answers with the correct ones. How well did you do?
